In [110]:
import schedule
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
baseurl = 'https://www.smartshanghai.com/housing/apartments-rent'

In [157]:
def get_listing_id(n):
    Listing_id = []
    Price = []
    for page in range(n):        
        params = {'page': page}
        response = requests.get(baseurl,params)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            content = soup.find_all("div", class_ = 'cont')  
        else:
            print(response.status_code)
        
        for i in range(len(content)):
            Listing_id.append(content[i].find('div').attrs['data-listingid'])
            Price.append(''.join(contents[i].find('div',class_="price").text.strip().split()[1].split(',')))
    return pd.DataFrame({'listing_id': Listing_id, 'price': Price})

In [165]:
new = get_listing_id(1)

In [160]:
len(new)

15

In [172]:
ex = new[:1]

In [204]:
for list_id in ex.listing_id:
    response = requests.get(f'{baseurl}/{list_id}')
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")

    #each list's information
    try:
        detail = soup_info.find('label', text='Type:').find_next_sibling('div').get_text(strip=True)
    except IndexError:
        print(list_id)
    

    print(detail)

Apartments


In [203]:
detail[0].find('label', text='Type:')

<label>Type:</label>

In [184]:
for indx, j in enumerate(detail):
    print(indx)
    print(j.text.strip())

0
Apartments
1
Available Now
2
12 months
3
1 months' rent
4
1 months' rent
5
50%
6
2 Bedrooms, 1 Bathrooms
7
75 sqm
8
13
9
Furnished
10
South
11
Xuhui
12
Xujiahui
13
Zhaofeng Dijngyuan  /  兆丰帝景苑
14
5 minutes walk to Xujiahui on line(s)     
1


9


11
15
75 square meter of modern design and quality furniture , artistic paintings and green decorations 

whole house underfloor heating and central air conditioning

Close to Xujiahui park, good for a night run or morning walk , metro city mall nearby, easy for shopping or dining 

password lock for viewing at any time


In [185]:
for indx, j in enumerate(detail):
    print(indx)
    print(j.text.strip())

0
Apartments
1
Available Now
2
12 months
3
1 months' rent
4
1 months' rent
5
50%
6
2 Bedrooms, 1 Bathrooms
7
75 sqm
8
13
9
Furnished
10
South
11
Xuhui
12
Xujiahui
13
Zhaofeng Dijngyuan  /  兆丰帝景苑
14
5 minutes walk to Xujiahui on line(s)     
1


9


11
15
75 square meter of modern design and quality furniture , artistic paintings and green decorations 

whole house underfloor heating and central air conditioning

Close to Xujiahui park, good for a night run or morning walk , metro city mall nearby, easy for shopping or dining 

password lock for viewing at any time


In [16]:
def page_data(data):

    features = ['Type', 'Available From', 'Agency Commission', 'Rooms', 'Size',
               'Floor', 'Furnished', 'Main Window Facing', 'District', 'Area',
                'Compound', 'Metro Station', 'Longtitue', 'Latitude', 'posting agent', 'description', 'first_post', 'Refresh']
    

    for list_id in data.Listing_Id:
        response = requests.get(f'{baseurl}/{list_id}')
        if response.status_code == 200:
            soup_info = BeautifulSoup(response.content, "html.parser")

        
        #each list's information
        try:
            detail = soup_info.find_all('div', class_='details')[0].find_all(name='div')
        except IndexError:
            print(list_id)
            
        #from 'Type' to 'Area'
        for indx, j in enumerate(detail[0:-3]):
            house_data.loc[list_id,features[indx]] = j.text.strip()
        
        
        #'Compound'
        house_data.loc[list_id,"Compound"] = detail[-3].text.split('/')[0].strip()
        
        
        # metro station
        text = detail[-2].text 
        try:
            found = re.search('walk to(.+?)on line', text).group(1)
        except AttributeError:
            found = ''
        house_data.loc[list_id,"Metro"] = found.strip()
        
        #long & lat
        long = soup_info.find('span', itemprop="longitude").text
        lat = soup_info.find('span', itemprop="latitude").text
        house_data.loc[list_id,"Longtitude"] = long
        house_data.loc[list_id,"Latitude"] = lat
        
        #posting agent
        house_data.loc[list_id,"Agent"] = soup_info.find('p', class_='username').text
        
        #description
        house_data.loc[list_id,"Description"] = soup_info.find('div', class_='description').text.strip()
        
        #post and views
        post = soup_info.find('div', class_='posted-and-views').text.strip().split(',')
        
        house_data.loc[list_id,"First_post"] = ' '.join(post[0].split(' ')[1:])
        house_data.loc[list_id,"Refresh"] = ' '.join(post[2].split(' ')[2:])
        
        #values.append(value)  # all listings
        
        
        #amenities 
        amenity_pos = soup_info.find('div', class_='amenities').find_all('li', class_='positive')
        amenity_neg = soup_info.find('div', class_='amenities').find_all('li', class_='negative')
        
        amenity_pos = [i.text.strip() for i in amenity_pos]
        amenity_neg = [i.text.strip() for i in amenity_neg]
        
        for indx, amenity in enumerate(amenity_pos):
            house_data.loc[list_id,amenity_pos[indx]] = 1
        
        for indx, amenity in enumerate(amenity_neg):
            house_data.loc[list_id, amenity_neg[indx]] = 0

        
    return house_data

In [58]:
def details(raw):
    for list_id in raw['Listing_Id']:
        response = requests.get(f'{baseurl}/{list_id}')
        if response.status_code == 200:
            soup_info = BeautifulSoup(response.content, "html.parser")
            amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])

        amenities_dict = {}
        for amenity_li in amenities:
            amenity = amenity_li.text.strip()
            amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0

                # Add the amenity to the corresponding row in the original DataFrame
        raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])
        #raw.loc[raw['Listing_Id'] == list_id, amenity_name] = status
    return raw

In [59]:
details(raw)

/tmp/ipykernel_6435/148635612.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])


,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
1,1605144,11000,60,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
requests.get(f'{baseurl}/1605144').content

b'<!doctype html>\n<html lang="en">\n<head prefix="og: http://ogp.me/ns#">\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />\n    <title>Impressive 1BR lane house | Jing\'an Temple L2/7 | Highceiling | Terrace | SmartShanghai</title>\n    \n    <meta property="fb:app_id" content="1333356330341868"/>\n<meta property="og:type" content="website"/>\n<meta property="og:url" content="https://www.smartshanghai.com/housing/apartments-rent/1605144"/>\n<meta property="og:site_name" content="SmartShanghai.com"/>\n<meta property="og:title" content="Impressive 1BR lane house | Jing\'an Temple L2/7 | Highceiling | Terrace"/>\n<meta property="og:description" content="Layout: 1 Br/1 Bt Floor: 3F Price: RMB 11000 Metro: 10 minutes walking to L2/11 at Jiangsu rd metro station. 5 minutes walking to L2/7 at Jing&rsquo;a..."/>\n<meta name="description" content="Layout: 1 Br/1 Bt Floor: 3F Price: RMB 11000 Metro: 10

In [87]:
response = requests.get(f'{baseurl}/1605144')

In [88]:
soup_info = BeautifulSoup(requests.get(f'{baseurl}/1605144').content, "html.parser")

In [89]:
detail = soup_info.find_all('div', class_='details')[0].find_all(name='div')

In [105]:
detail

[<div>
                                                                                     Apartments
                                         
                                                                             </div>,
 <div>
                                         Available Now                                    </div>,
 <div>
                                         12 months                                     </div>,
 <div>
 <b>1</b> months' rent                                    </div>,
 <div>
 <b>3</b> months' rent                                    </div>,
 <div>
                                         35%
                                     </div>,
 <div>
 <span itemprop="numberOfBedrooms">1</span> Bedrooms, <span itemprop="numberOfBathroomsTotal">1</span> Bathrooms
                                         </div>,
 <div>
                                         60 sqm                                    </div>,
 <div>
                                         3
   

In [109]:
for indx, j in enumerate(detail):
    print(indx)
    print(j.text.strip())
    

0
Apartments
1
Available Now
2
12 months
3
1 months' rent
4
3 months' rent
5
35%
6
1 Bedrooms, 1 Bathrooms
7
60 sqm
8
3
9
Furnished
10
South
11
Jing'an
12
Jing'an Temple
13
Bubbling Well Lane  /  涌泉坊
14
5 minutes walk to Jiangsu Rd on line(s)     
11


2
15
Layout: 1 Br/1 Bt
Floor: 3F
Price: RMB 11000
Metro: 10 minutes walking to L2/11 at Jiangsu rd metro station.
5 minutes walking to L2/7 at Jing’an Temple metro station .

Nice, clean modern style furniture, loft style.
Well Decorated with modern fresh style.
Fully equipped with kitchen, clean bathroom.bright and cozy home feeling .
Whole 3rd floor ,with a lot of storage .
Very high ceiling ,floor heating ,central AC .
Public terrace on the top .

You can spend your free time here reading, exercising, having dinners with friends etc. And taking care of flowers and plants if you enjoy that.

Super competitive price arrcoding to the decoration and the location .


Can view at anytime - mornings, evenings, weekends.


In [91]:
Amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])

In [93]:
amenities_dict = {}
for amenity_li in Amenities:
    amenity = amenity_li.text.strip()
    amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
            
            # Add the amenity to the corresponding row in the original DataFrame
raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])

In [94]:
amenities_dict

{'Air Filter': 0,
 'Balcony': 1,
 'English Speaking Landlord': 0,
 'Fitness Centers': 0,
 'Floor Heating': 1,
 'Garden': 0,
 'Historic Building': 0,
 'Landlord lives in Shanghai': 1,
 'Large Storage Room': 1,
 'Oven': 1,
 'Parking': 0,
 'Pets Allowed\n\n                                                                                    \n                                                true': 1,
 'Playground': 0,
 'Pool': 0,
 'Tennis Courts': 0,
 'Villa': 0,
 'Wall heating': 0,
 'Water Filter': 0}

In [97]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms
1,1605144,11000,60,1,1


In [98]:
pd.DataFrame([amenities_dict])

,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,Garden,Historic Building,Landlord lives in Shanghai,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
0,0,1,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0


In [96]:
raw.merge(pd.DataFrame([amenities_dict]), left_index=True, right_index=True)

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter


In [73]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
1,1605144,11000,60,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
amenities = []
for li in Amenities:
    amenity = li.text.strip()
    status = 1 if 'positive' in li['class'] else 0
    amenities.append({'amenity': amenity, 'status': status})


In [80]:
amenities

[{'amenity': 'Air Filter', 'status': 0},
 {'amenity': 'Balcony', 'status': 1},
 {'amenity': 'English Speaking Landlord', 'status': 0},
 {'amenity': 'Fitness Centers', 'status': 0},
 {'amenity': 'Floor Heating', 'status': 1},
 {'amenity': 'Garden', 'status': 0},
 {'amenity': 'Historic Building', 'status': 0},
 {'amenity': 'Landlord lives in Shanghai', 'status': 1},
 {'amenity': 'Large Storage Room', 'status': 1},
 {'amenity': 'Oven', 'status': 1},
 {'amenity': 'Parking', 'status': 0},
 {'amenity': 'Pets Allowed\n\n                                                                                    \n                                                true',
  'status': 1},
 {'amenity': 'Playground', 'status': 0},
 {'amenity': 'Pool', 'status': 0},
 {'amenity': 'Tennis Courts', 'status': 0},
 {'amenity': 'Villa', 'status': 0},
 {'amenity': 'Wall heating', 'status': 0},
 {'amenity': 'Water Filter', 'status': 0}]

In [48]:
for list_id in raw['Listing_Id']:
    response = requests.get(f'{baseurl}/{list_id}')
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")
        amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
        
    amenities_dict = {}
    for amenity_li in amenities:
        amenity = amenity_li.text.strip()
        amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
            
    # Add the amenity to the corresponding row in the original DataFrame
    raw.loc[raw['Listing_Id'] == list_id, list(amenities_dict.keys())] = pd.DataFrame([amenities_dict])
    #raw.loc[raw['Listing_Id'] == list_id, amenity_name] = status

In [49]:
raw

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Air Filter,Balcony,English Speaking Landlord,Fitness Centers,Floor Heating,...,Large Storage Room,Oven,Parking,Pets Allowed\n\n \n true,Playground,Pool,Tennis Courts,Villa,Wall heating,Water Filter
0,1605145,10500,85,2,1,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1605144,11000,60,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
